# Devoir 2
rédigé par Corey Ducharme et Olivier Sirois

## Préamble

In [7]:
using JuMP
using Clp

Considérons le problème d'allocation de la demande stochastique.

On défini nos variables  

$$ X = (x_1, x_2, x_3, x_4)  \\
B = (10, 7, 16, 6) \\
Y = \begin{pmatrix}y_{11} & y_{12} & y_{13} \\ y_{21} & y_{22} & y_{23} \\ y_{31} & y_{32} & y_{33} \\ y_{41} & y_{42} & y_{43}\end{pmatrix} \\
A = \begin{pmatrix}40 & 24 & 4 \\ 45 & 27 & 4.5 \\ 32 & 19.2 & 3.2 \\ 55 & 33 & 5.5\end{pmatrix} $$

où $x_i$ représente la capacité de production de l'usine i,

B le vector des coûts de production,

$y_{ij}$ la production de glace j dans l'usine i,

$a_{ij}$ le cout de production pour la glace j dans l'usine i.

Nous posons maintenant le problème d'optimisation.

\begin{align*}
    \min_x\ & BX^{\intercal} + E[Q(x)]\\
    \mbox{s.t. } & X \geq \hat{0} \\
     & BX^{\intercal} \leq 120 \\
     & Xe \geq 12
\end{align*}


où le problème de 2e étape est 

\begin{align*}
Q(x) = \min_y\ & e^{\intercal}A \circ Y e \\
\mbox{s.t. } & Ye \leq X^{\intercal} \\
& Y^{\intercal}e \geq (\xi, 3, 2)^{\intercal}
\end{align*}

ici $\circ$ est le produit d'Hadamard et les opérations d'inégalités ce font éléments par éléments entre deux vecteurs. $e$ est un vecteur colonne unitaire associée à la matrice le précédent.

In [8]:
# Define the variables
scenarios = 3
p = [0.3, 0.4, 0.3]
ξ = [3, 5, 7]
B = [10 7 16 6]
A = [40 24 4; 45 27 4.5; 32 19.2 3.2; 55 33 5.5]
#theta = -Inf

-Inf

In [9]:
function master(theta)
    master = Model(solver=ClpSolver())
    
    @variable(master, x[1:4] >= 0)
    
    @constraint(master, sum(x[i]*B[i] for i=1:4) <= 120)
    @constraint(master, sum(x[i] for i = 1:4) >= 12)
    
    @objective(master, Min, sum(x[i]*B[i] for i=1:4) + theta)
    
    status = solve(master)
    
    return master, status, getvalue(x)
end

master (generic function with 1 method)

In [10]:
test, stat, x = master(-Inf)

x

4-element Array{Float64,1}:
  0.0
  0.0
  0.0
 12.0

In [60]:
function secondstage(x::Vector, ξ::Int)
    m = Model(solver=ClpSolver())    

    @variable(m, y[1:4,1:3] >= 0)
    
    @constraintref artCons[1:7]

    #NOTE : est-ce que la formule du dual va quand meme bien fonctionner si on met notre contrainte comme 
    #       etant un 'greater then' au lieu d'un 'lesser then' (>= vs. <=) dans le cas de nos contraintes
    #       1,2,3 ??
    artCons[1] = @constraint(m, sum(y[i,1] for i=1:4) >= ξ)
    artCons[2] = @constraint(m, sum(y[i,2] for i=1:4) >= 3)
    artCons[3] = @constraint(m, sum(y[i,3] for i=1:4) >= 2)
    artCons[4] = @constraint(m, sum(y[1,j] for j=1:3) <= x[1])
    artCons[5] = @constraint(m, sum(y[2,j] for j=1:3) <= x[2])
    artCons[6] = @constraint(m, sum(y[3,j] for j=1:3) <= x[3])
    artCons[7] = @constraint(m, sum(y[4,j] for j=1:3) <= x[4])
    
    
    #for i=1:4
    #    @constraint(m, sum(y[i,j] for j=1:3) <= x[i])
    #end
    
    @objective(m, Min, sum(A[i,j]*y[i,j] for i=1:4, j=1:3))
    
    status = solve(m)
    
    dual = getdual(artCons)
    
    
    return m, status, getvalue(y), dual
end

function secondstage_feasability(x::Vector, ξ::Int)
    m = Model(solver = ClpSolver())

    @variable(m, y[1:4,1:3] >= 0)
    @variable(m, splus[1:7] >= 0)
    @variable(m, sminus[1:7] >= 0)

    @constraintref artCons[1:7]
    
    artCons[1] = @constraint(m, sum(y[i,1] for i=1:4) + splus[1] - sminus[1] >= ξ)
    artCons[2] = @constraint(m, sum(y[i,2] for i=1:4) + splus[2] - sminus[2] >= 3)
    artCons[3] = @constraint(m, sum(y[i,3] for i=1:4) + splus[3] - sminus[3] >= 2)
    artCons[4] = @constraint(m, sum(y[1,j] for j=1:3) + splus[4] - sminus[4] <= x[1])
    artCons[5] = @constraint(m, sum(y[2,j] for j=1:3) + splus[5] - sminus[5] <= x[2])
    artCons[6] = @constraint(m, sum(y[3,j] for j=1:3) + splus[6] - sminus[6] <= x[3])
    artCons[7] = @constraint(m, sum(y[4,j] for j=1:3) + splus[7] - sminus[7] <= x[4])
    

    @objective(m,Min, sum(splus[i] for i=1:7)+sum(sminus[i] for i=1:7))
    
    status = solve(m)

    dual = getdual(artCons)

    return m, status, getobjectivevalue(m), dual
end

secondstage_feasability (generic function with 1 method)

In [12]:
martificial, status, y, dual = secondstage([0,0,0,12], 3)

(Minimization problem with:
 * 7 linear constraints
 * 12 variables
Solver is ClpMathProg, :Optimal, [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 3.0 3.0 2.0], [55.0, 33.0, 5.5, -15.0, -10.0, -23.0, 0.0])

In [13]:
print(martificial)

Min 40 y[1,1] + 24 y[1,2] + 4 y[1,3] + 45 y[2,1] + 27 y[2,2] + 4.5 y[2,3] + 32 y[3,1] + 19.2 y[3,2] + 3.2 y[3,3] + 55 y[4,1] + 33 y[4,2] + 5.5 y[4,3]
Subject to
 y[1,1] + y[2,1] + y[3,1] + y[4,1] ≥ 3
 y[1,2] + y[2,2] + y[3,2] + y[4,2] ≥ 3
 y[1,3] + y[2,3] + y[3,3] + y[4,3] ≥ 2
 y[1,1] + y[1,2] + y[1,3] ≤ 0
 y[2,1] + y[2,2] + y[2,3] ≤ 0
 y[3,1] + y[3,2] + y[3,3] ≤ 0
 y[4,1] + y[4,2] + y[4,3] ≤ 12
 y[i,j] ≥ 0 ∀ i ∈ {1,2,3,4}, j ∈ {1,2,3}


In [14]:
y

4×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 3.0  3.0  2.0

In [15]:
getobjectivevalue(martificial)

275.0

In [16]:
dual

7-element Array{Float64,1}:
  55.0
  33.0
   5.5
 -15.0
 -10.0
 -23.0
   0.0

In [17]:
function optimalcut(x::Vector, ξ::Int, p::Float64, dual::Vector)
    h =  [ξ; 3; 2; 0; 0; 0; 0];
    T =  [0 0 0 0; 0 0 0 0; 0 0 0 0; -x[1] 0 0 0; 0 -x[2] 0 0; 0 0 -x[3] 0; 0 0 0 -x[4]]   
    small_e = p * dot(vec(dual), h)
    E = zeros(4)
    for i = 1:4
        E[i] = p* dot(vec(dual),T[:,i])
    end
    return small_e, E
end

function feascut(x::Vector, ξ::Int, dual::Vector)
    h =  [ξ; 3; 2; 0; 0; 0; 0];
    T =  [0 0 0 0; 0 0 0 0; 0 0 0 0; -x[1] 0 0 0; 0 -x[2] 0 0; 0 0 -x[3] 0; 0 0 0 -x[4]] 
    small_sig = dot(vec(dual), h)
    big_sig = zeros(4)
    for i = 1:4
        big_sig[i] = dot(vec(dual), T[:,i])
    end
    return small_sig, big_sig
    
end

feascut (generic function with 1 method)

In [51]:
e, E = feascut([0,0,0,12], 3, dual)
@printf("petit e = %f;\n",e)
E

#@constraint(?master_model?, cut[counter++], sum(E[i]*x[i] for i = 1:2) >= e)

petit e = 0.000000;


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [69]:
##SKELETON

#Transcription du pseudocode du Kall+Wallace, il nous reste qu'a implementer l'algorithme maitre

K = 0
L = 0
stop = 0
p = [0.3 0.4 0.3]
realisation = [3 5 7]
#@constraintref masterconstraints[1:1000]
optconstraintcounter = 1
feasconstraintcounter = 1

mas = Model(solver=ClpSolver())

@variable(mas, x[1:4] >= 0)
@variable(mas, theta >= -9999999999999999)
@constraint(mas, sum(x[i]*B[i] for i=1:4) <= 120)
@constraint(mas, sum(x[i] for i = 1:4) >= 12)
    
@objective(mas, Min, sum(x[i]*B[i] for i=1:4) + theta)
    
status = solve(mas)
Q_rond = getobjectivevalue(mas)
a = getvalue(x)

@printf("%f\n",Q_rond)
if (status == :Infeasible)
    stop = 1
end
while (stop == 0)
    
    
    ### On fait une coupe de faisabiliter si necessaire et on recommence jusqu'a temps qu'on ne peut plus. sinon
    x_mod = getvalue(x)
    small_sig = 0
    big_sig = [0,0,0,0]
    obj = 0
    for i = 1:3
        m, status, temp_obj, dual = secondstage_feasability(x_mod, realisation[i])
        temp_small_sig, temp_big_sig = feascut(x_mod, realisation[i], dual)
        
        #println(temp_small_sig)
        #println(temp_big_sig)
        obj += temp_obj
        small_sig += temp_small_sig
        big_sig += temp_big_sig  
    end
    
    if(obj == 0)
        newcut = 0
    else
        newcut = 1
        @constraint(mas, cut[constraintcounter], sum(big_sig[i]*x[i] for i = 1:4) >= small_sig)
        constraintcounter += 1
    end    
    
    if(newcut == 0)
        #on commence nos coupes d'optimalit/s
        E = [0,0,0,0]
        e = 0
        
        #on accumule E et e pour tous les realisation (pour faire la sommation)
        for i = 1:3
            m, status, y, dual = secondstage(x_mod, realisation[i])
            temp_e, temp_E = optimalcut(x_mod, realisation[i], p[i], dual)
            E += temp_E
            e += temp_e    
        end
        
        #on calcule notre w
        w = e - dot(E, x_mod)
    
        #si il est plus petit que la valeur objective, on arrete
        if (w < getobjectivevalue(mas))
            stop = 1
        else
            #sinon, on rajoute une coupe
            @constraint(mas, cut[constraintcounter], sum(E[i]*x[i] for i = 1:4) + theta >= e)
            constraintcounter += 1
        end
    
        #dans le cas ou le stop n'est pas egal a un, on resous notre nouveau probleme avec la nouvelle contrainte,
        #s'il devient infaisaible, on arrete
        if (stop != 1)
            status = solve(mas)
            Q_rond = getobjectivevalue(mas)
            if(status == :Infeasible)
                stop = 1
            end
            println(Q_rond)
        end
    
    
    #print(E)
    #print(e)
    #if(?No New Feascut?)
    
        #On cherche pour la valeur de notre model -> Q_rond 
    
        #if(theta >= Q_rond(x))
            #stop = 1
        #else
            
            #on trouve notre coupe d'optimaliter et on l'applique
    
        #end
    
        #if(stop != 1)
    
            #on resoud le probleme maitre, si il est infaisable,
    
            #if(status == infeasible)
    
                #stop == 1
    
            #end
    
        #end
    
    end
    #end
    
    #WARNING, OVERRIDE DU LOOP POUR PAS QU'IL RESTE POGNER DEDANS
    stop = 1
end

-9999999999999928.000000
457.0
